# Instalação

In [1]:
pip install PyMuPDF


   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.2 MB 3.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.8/3.2 MB 2.8 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.2 MB 2.2 MB/s eta 0:00:01
   ---------------- ----------------------- 1.3/3.2 MB 1.6 MB/s eta 0:00:02
   ---------------- ----------------------- 1.3/3.2 MB 1.6 MB/s eta 0:00:02
   ------------------- -------------------- 1.6/3.2 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.6/3.2 MB 1.2 MB/s eta 0:00:02
   ---------------------- ----------------- 1.8/3.2 MB 1.0 MB/s eta 0:00:02
   ---------------------- ----------------- 1.8/3.2 MB 1.0 MB/s eta 0:00:02
   ------------------------- -------------- 2.1/3.2 MB 954.7 kB/s eta 0:00:02
   ----------------------------- ---------- 2.4/3.2 MB 938.7 kB/s eta 0:00:01
   ----------------------------- ---------- 2.4/3.2 MB 938.7 kB/s eta 0:00:01
   ---------

# Import

In [2]:
import fitz  # PyMuPDF
import re

# Funções

In [64]:
def extract_text_from_pdf(pdf_path, page_numbers):
    """
    Extrai texto de páginas específicas de um arquivo PDF.

    Args:
    pdf_path (str): Caminho para o arquivo PDF.
    page_numbers (list): Lista de números de páginas a serem extraídas (baseado em 1).

    Returns:
    dict: Dicionário com números de páginas como chaves e textos extraídos como valores.
    """
    try:
        pdf_document = fitz.open(pdf_path)
    except Exception as e:
        print(f"Erro ao abrir o PDF: {e}")
        return {}
    
    # Inicializa um dicionário para armazenar os textos extraídos
    extracted_texts = {}
    
    # Itera sobre os números das páginas fornecidos
    for page_number in page_numbers:
        # Ajusta o número da página (PyMuPDF usa índice baseado em 0)
        page_index = page_number - 1
        
        # Verifica se a página existe
        if page_index < len(pdf_document):
            # Obtém a página
            page = pdf_document[page_index]
            
            blocks = page.get_text("blocks")  # Extrair texto por blocos
            
            # Separar os blocos em colunas
            first_column_blocks = [block for block in blocks if block[0] < page.rect.width / 2]
            second_column_blocks = [block for block in blocks if block[0] >= page.rect.width / 2]
            
            # Ordenar os blocos de cada coluna pela posição Y
            first_column_blocks = sorted(first_column_blocks, key=lambda b: b[1])
            second_column_blocks = sorted(second_column_blocks, key=lambda b: b[1])
            
            # Concatenar os blocos ordenados de cada coluna
            text = " ".join(block[4] for block in first_column_blocks) + " " + " ".join(block[4] for block in second_column_blocks)
            
            
            # Armazena o texto extraído no dicionário
            extracted_texts[page_number] = text
        else:
            print(f"A página {page_number} não existe no documento.")
    
    # Fecha o documento PDF
    pdf_document.close()
    
    return extracted_texts


In [65]:
def preprocess_text(text):
    """
    Pré-processa o texto removendo quebras de linha, múltiplos espaços e pontos duplicados.

    Args:
    text (str): Texto a ser pré-processado.

    Returns:
    str: Texto pré-processado.
    """
    # Remove quebras de linha e substitui múltiplos espaços por um único espaço
    text = re.sub(r"\s+", " ", text)
    
    # Remove pontos duplicados para simplificar a identificação das entradas
    text = re.sub(r"\.{3,}", "...", text)
    
    # Corrige palavras hifenizadas no final da linha
    text = re.sub(r'-\s+', '', text)
    
    return text

In [66]:
def parse_summary(summary_text, page_not_list=0):
    """
    Converte o texto do sumário em um dicionário estruturado.

    Args:
    summary_text (str): Texto do sumário.
    page_not_list (int): Quantidade de páginas não listadas no PDF.

    Returns:
    dict: Dicionário com títulos como chaves e números de páginas ajustados como valores.
    """
    # Inicializa um dicionário para armazenar o sumário estruturado
    summary_dict = {}
    
    # Define um padrão de regex para capturar cada entrada do sumário
    entry_pattern = re.compile(r"(.+?)\s*\.{3,}\s*(\d+)")
    
    # Encontra todas as entradas do sumário
    entries = entry_pattern.findall(summary_text)
    
    # Preenche o dicionário com as entradas do sumário
    for entry in entries:
        title, page_number = entry
        title = title.strip().rstrip('.')
        summary_dict[title.strip()] = int(page_number) + page_not_list
    
    return summary_dict

In [67]:
def extract_info_from_page(pdf_path, summary_dict, title):
    """
    Extrai informações de uma página específica com base no título do sumário.

    Args:
    pdf_path (str): Caminho para o arquivo PDF.
    summary_dict (dict): Dicionário do sumário estruturado.
    title (str): Título a ser buscado no sumário.

    Returns:
    dict or str: Dicionário com o título e o texto extraído ou mensagem de erro se o título não for encontrado.
    """
    if title not in summary_dict:
        return f"O título '{title}' não foi encontrado no sumário."

    titles = list(summary_dict.keys())
    title_index = titles.index(title)
    start_page = summary_dict[title]
    end_page = summary_dict[titles[title_index + 1]] if title_index + 1 < len(titles) else None

    if end_page:
        page_numbers = list(range(start_page, end_page+1))
    else:
        try:
            pdf_document = fitz.open(pdf_path)
            total_pages = len(pdf_document)
            page_numbers = list(range(start_page, total_pages + 1))
            pdf_document.close()
        except Exception as e:
            return f"Erro ao abrir o PDF: {e}"

    extracted_texts = extract_text_from_pdf(pdf_path, page_numbers)
    
    combined_text = ""
    found_title = False
    for page in page_numbers:
        page_text = extracted_texts[page]
        if not found_title:
            title_start_index = page_text.find(title)
            if title_start_index != -1:
                found_title = True
                combined_text += page_text[title_start_index:]
            else:
                continue
        else:
            combined_text += page_text

    # Corte o texto no próximo título, se houver
    if title_index + 1 < len(titles):
        next_title = titles[title_index + 1]
        next_title_index = combined_text.find(next_title)
        if next_title_index != -1:
            combined_text = combined_text[:next_title_index]

    return {title: preprocess_text(combined_text)}

In [68]:
def extract_relevant_info(text, keywords):
    """
    Extrai informações relevantes do texto baseado em keywords específicas.

    Args:
    text (str): Texto completo onde as informações serão extraídas.
    keywords (list): Lista de strings contendo as palavras-chave a serem procuradas.

    Returns:
    dict: Dicionário com as keywords como chaves e o texto encontrado até a próxima keyword como valores.
    """
    relevant_info = {}
    current_key = None
    text_lower = text.lower()  # Convertendo para minúsculas para facilitar a busca

    for keyword in keywords:
        keyword_lower = keyword.lower()
        match = re.search(re.escape(keyword_lower), text_lower)
        if match:
            start_index = match.start()
            if current_key:
                relevant_info[current_key] = text[current_key_start:start_index].strip()
            current_key = keyword
            current_key_start = start_index

    if current_key:
        relevant_info[current_key] = text[current_key_start:].strip()

    return relevant_info

In [69]:
def process_formatted_info(formatted_info):
    """
    Processa o dicionário formatted_info para criar sub-dicionários com base nas keywords fornecidas.

    Args:
    formatted_info (dict): Dicionário com a estrutura original.
    keywords (list): Lista de strings contendo as palavras-chave a serem procuradas.

    Returns:
    dict: Dicionário com a mesma estrutura do formatted_info, mas com sub-dicionários baseados nas keywords.
    """
    processed_info = {}
    
    for main_key, sections in formatted_info.items():
        keywords = ["Pontos de Vida.", 
        "Pontos de Mana.", 
        "Perícias.",
        "Proficiências.",
        f"Caminho do {main_key}.", 
        "Magias.", 
        f"Poder de {main_key}.", 
        "Alta Arcana.",
        "Linhagem Dracônica",
        "Linhagem Feérica",
        "Linhagem Rubra",
        "Habilidades de Raça",
        "Versátil."]
        
        processed_info[main_key] = {}
        for section_title, text in sections.items():
            relevant_info = extract_relevant_info(text, keywords)
            processed_info[main_key][section_title] = relevant_info

    return processed_info

In [70]:
def clean_key(data):
    """Remove the key name from the text if it matches the beginning of the value."""
    cleaned_data = {}
    for key, value in data.items():
        if isinstance(value, dict):
            cleaned_data[key] = clean_key(value)
        elif isinstance(value, str):
            first_sentence = value.split('.', 1)[0].strip()
            if key.strip('.') == first_sentence:
                cleaned_data[key] = value[len(first_sentence) + 1:].strip()
            else:
                cleaned_data[key] = value
        else:
            cleaned_data[key] = value
    return cleaned_data

In [71]:
def extract_subsections(processed_info):
    def process_text(text):
        """Helper function to process text and extract subsections."""
        sections = text.split('•')
        main_content = sections[0].strip()
        subsections = {'main': main_content}
        for section in sections[1:]:
            parts = section.split('.', 1)
            if len(parts) > 1:
                title = parts[0].strip()
                content = parts[1].strip()
                subsections[title] = content
        return subsections

    def recursive_process(data):
        """Recursively process the dictionary."""
        if isinstance(data, dict):
            processed = {}
            for key, value in data.items():
                if isinstance(value, str) and '•' in value:
                    processed[key] = process_text(value)
                else:
                    processed[key] = recursive_process(value)
            return processed
        else:
            return data

    new_info = recursive_process(processed_info)
    cleaned_info = clean_key(new_info)
    return cleaned_info


# Execução

In [72]:
# Exemplo de uso
pdf_path = './data/jamboeditora-t20-jogo-do.pdf'
page_numbers = [9]  # Página que deseja extrair
page_not_list = 6 # Quantidade de paginas não listatadas no pdf
# Lista de keywords para identificar seções relevantes

keywords = [
    "Habilidades de Raça",
    "Características de Classe",
    "Habilidades de Classe",
    "Linhagens Sobrenaturais"
]


In [73]:
# Passo 1: Extrair o texto da página
summary_text = extract_text_from_pdf(pdf_path, page_numbers)

# Verifica se o sumário foi extraido
if not summary_text:
        print("Erro ao extrair o texto do sumário.")

# Passo 2: Pré-processar o sumário
processed_summary = preprocess_text(summary_text[page_numbers[0]])

# Passo 3: Estruturar o sumário como um dicionário
summary_dict = parse_summary(processed_summary, page_not_list)

# Sumário Estruturado
summary_dict

{'Sumário Prefácio': 10,
 'introdução': 12,
 'Capítulo 2: Perícias & Poderes': 118,
 'Perícias': 120,
 'Acrobacia': 121,
 'Adestramento': 121,
 'Atletismo': 122,
 'Atuação': 122,
 'Cavalgar': 122,
 'Conhecimento': 123,
 'Cura': 123,
 'Diplomacia': 124,
 'Enganação': 124,
 'Fortitude': 125,
 'Furtividade': 125,
 'Guerra': 125,
 'Iniciativa': 125,
 'Intimidação': 126,
 'Intuição': 126,
 'Investigação': 126,
 'Jogatina': 126,
 'Ladinagem': 126,
 'Luta': 127,
 'Misticismo': 127,
 'Nobreza': 127,
 'Ofício': 127,
 'Percepção': 128,
 'Pilotagem': 128,
 'Pontaria': 128,
 'Reflexos': 128,
 'Religião': 128,
 'Sobrevivência': 129,
 'Vontade': 129,
 'Poderes Gerais': 130,
 'Poderes de Combate': 130,
 'Poderes de Destino': 135,
 'Poderes de Magia': 137,
 'Poderes Concedidos': 138,
 'Poderes da Tormenta': 142,
 'O que é Tormenta20?': 14,
 'Mecânica Básica': 15,
 'Termos Importantes': 17,
 '20 Coisas a Saber': 18,
 'Capítulo 1: construção de personagem': 20,
 'Conceito de Personagem': 22,
 'Atributos

In [74]:
# Passo 4: Extrair informações de uma página específica com base no dicionário
title_to_extract = 'Arcanista'
extracted_info = extract_info_from_page(pdf_path, summary_dict, title_to_extract)

# Exibir informações extraídas
print("Informações extraídas da página correspondente ao título:")
print(extracted_info)

Informações extraídas da página correspondente ao título:
{'Arcanista': 'Arcanista A magia é a força mais poderosa de Arton. Está presente em todas as grandes maravilhas deste mundo, impregnada nas muralhas e torres dos maiores castelos e masmorras. Criaturas fantásticas são tocadas pela magia, armas e artefatos lendários são imbuídos de poder mágico. Mesmo assim, a magia permanece um mistério. Ninguém pode dizer que compreende totalmente esta força caprichosa, imprevisível, devastadora e deslumbrante. A magia esconde segredos infinitos, desde o truque de um ilusionista de rua até o poder de uma bola de fogo, desde o encanto para aprimorar uma espada até o segredo de cruzar dimensões. Esta busca é constante, pois a disciplina da magia exige dedicação total. Em início de carreira, os arcanistas costumam ser fracos, frágeis, quase indefesos. Contudo, à medida que sua experiência aumenta, logo se tornam oponentes formidáveis. Em vez de serem protegidos por seus aliados, tomam para si o pa

In [75]:
# Passo 5: Extrair informações relevantes
formatted_info = {}
for section_title, text in extracted_info.items():
    relevant_info = extract_relevant_info(text, keywords)
    formatted_info[section_title] = relevant_info

formatted_info

{'Arcanista': {'Características de Classe': 'Características de Classe Pontos de Vida. Um arcanista começa com 8 pontos de vida (+ Constituição) e ganha 2 PV (+ Constituição) por nível. Pontos de Mana. 6 PM por nível. Perícias. Misticismo (Int) e Vontade (Sab), mais 2 a sua escolha entre Conhecimento (Int), Diplomacia (Car), Enganação (Car), Guerra (Int), Iniciativa (Des), Intimidação (Car), Intuição (Sab), Investigação (Int), Nobreza (Int), Ofício (Int) e Percepção (Sab). Proficiências. Nenhuma.',
  'Habilidades de Classe': 'Habilidades de Classe Caminho do Arcanista. A magia é um poder incrível, capaz de alterar a realidade. Esse poder tem fontes distintas e cada uma opera conforme suas próprias regras. Escolha uma das opções a seguir. Uma vez feita, essa escolha não pode ser mudada. • Bruxo. Você lança magias através de um foco — uma varinha, cajado, chapéu... Para lançar uma magia, você precisa empunhar o foco com uma mão (e gesticular com a outra) ou fazer um teste de Misticismo (

In [76]:
# Passo 6: Formatar informações
processed_info = process_formatted_info(formatted_info)
new_processed_info = extract_subsections(processed_info)
new_processed_info

{'Arcanista': {'Características de Classe': {'Pontos de Vida.': 'Um arcanista começa com 8 pontos de vida (+ Constituição) e ganha 2 PV (+ Constituição) por nível.',
   'Pontos de Mana.': '6 PM por nível.',
   'Perícias.': 'Misticismo (Int) e Vontade (Sab), mais 2 a sua escolha entre Conhecimento (Int), Diplomacia (Car), Enganação (Car), Guerra (Int), Iniciativa (Des), Intimidação (Car), Intuição (Sab), Investigação (Int), Nobreza (Int), Ofício (Int) e Percepção (Sab).',
   'Proficiências.': 'Nenhuma.'},
  'Habilidades de Classe': {'Caminho do Arcanista.': {'main': 'Caminho do Arcanista. A magia é um poder incrível, capaz de alterar a realidade. Esse poder tem fontes distintas e cada uma opera conforme suas próprias regras. Escolha uma das opções a seguir. Uma vez feita, essa escolha não pode ser mudada.',
    'Bruxo': 'Você lança magias através de um foco — uma varinha, cajado, chapéu... Para lançar uma magia, você precisa empunhar o foco com uma mão (e gesticular com a outra) ou faze